Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 '10X_PBMC']

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [4]:
# df = pd.read_pickle(f"{path}output/pickle_results/real_data/real_data_1model.pkl")

In [5]:
# df =df[df["temperature"]!= 0.008]

In [6]:
df = pd.DataFrame()
dropout = 0.9
lr = 1e-5
layers = [200, 40, 60]
# layers = [50]
temperature = 0.07
nb_genes = 500
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])
    cluster_number = np.unique(Y).shape[0]
    X = train.preprocess(X, nb_genes=nb_genes)
    for nb_epochs in [5, 10, 20, 30, 50, 100]:
        for run in range(3):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            dresults = train.run(X,
                                 cluster_number,
                                 dataset,
                                 Y=Y,
                                 nb_epochs=nb_epochs,
                                 lr=lr,
                                 temperature=temperature,
                                 dropout=dropout,
                                 evaluate=False,
                                 n_ensemble=1,
                                 layers=layers,
                                 save_to=f"{path}output/real_data/{run}/",
                                 save_pred = False)
            dresults["temperature"] = temperature
            dresults["dropout"] = dropout
            dresults["nb_epochs"] = nb_epochs
            dresults["layers"] = str(layers)
            dresults["run"] = run
            print(f".", end = "")
            print(f"# {temperature}, {dropout}, {lr}, {layers}", 
                  dresults.get('COMBINED_kmeans_ari', ""),
                  dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
                  dresults.get('leiden_ari_0', ""))
            df = df.append(dresults, ignore_index=True)

            df.to_pickle(f"{path}output/pickle_results/real_data/real_data_nb_epochs.pkl")

>>>>> Data Quake_Smart-seq2_Trachea
SCZI  0.8291128219663276
(1350, 23341) (1350, 23341) keeping 500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7679321465098168 0.17546034160699608
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.413289881678159 0.15975994184406833
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.43071508049582846 0.15266342837931543
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7964108740415752 0.18782087307260684
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8353788738649977 0.1811125047137331
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8208769760839248 0.17822405621333165
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8159393919493816 0.1814763257612075
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7936934801858778 0.19179545306954676
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8398440576403592 0.1834611735016804
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8757419746828033 0.18813728466297122
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8231397433913237 0.1741258162593457
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8804301360194373 0.170

.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8162506730523417 0.47882900300764303
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8354645835327029 0.4362398039307548
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8502066802555978 0.4660617504661891
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7467455310901463 0.482251486768899
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8543619300898888 0.4873218500900485
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8491479034251973 0.46175711302326006
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8439675942631009 0.47665399268481473
>>>>> Data Quake_10x_Bladder
SCZI  0.983095549108186
(2500, 23341) (2500, 23341) keeping 500 genes
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7511098125766029 0.19525811800341156
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7500599665863562 0.19997363063627135
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7494572220463509 0.18655056271939813
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7551111646819757 0.18536185732890736
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7533093804725166 0.190219758129

.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.43176597825838015 0.23500867242693202
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.43510844941820165 0.1984754425966739
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7203434271989837 0.247931624458194
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7335640501465499 0.2385761843079057
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7179296231668297 0.3019677168917052
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7284388899982704 0.2824650881563508
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7354186431860008 0.28504750448831334
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7289856480038408 0.2581196957117909
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7476412357871565 0.32082116610627753
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7509087933917664 0.2518488711481828
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7421953262727977 0.2879980426643929
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7366468444259135 0.28750393092790777
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7386017660956901 0.279067869899728
.# 0.07, 0.9, 1e-05, 

In [ ]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")
sczi = sczi[sczi["dataset"].isin(files)]
sczi.mean()

In [ ]:
df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_nb_epochs.pkl")
df = df[df["dataset"].isin(files)]
df.groupby(["temperature", "layers", "dropout", "nb_epochs"])["kmeans_ari_0"].mean().unstack(["layers", "nb_epochs"]).T

In [ ]:
# df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "nb_epochs"])["kmeans_ari_0"].mean().unstack(["layers", "nb_epochs"]).max()

In [ ]:
# r = df[(df["layers"]=="[40]")
#        &(df["temperature"]==0.01)
#        &(df["lr"]==1e-5)
#        &(df["dropout"]==0.9)
#       ]
r = df[(df["layers"]=="[200, 100, 30, 30]")
       &(df["temperature"]==0.07)
       &(df["lr"]==1e-5)
       &(df["dropout"]==0.9)
      ]


r.mean()

In [ ]:
comb = pd.merge(r, sczi, on=["dataset", "run"])[[
    "dataset", "kmeans_ari_0", "ARI", "kmeans_nmi_0", "NMI"
]].rename(columns = {"kmeans_ari_0": "contrative-sc", "ARI": "sczi"})
comb

In [ ]:
comb =pd.melt(comb, id_vars=["dataset"], value_vars=["contrative-sc", "sczi"])

In [ ]:
comb.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot( data = comb, y="value", x = "variable")
plt.xticks(rotation = 90)

In [ ]:
sns.barplot(x = "dataset", data = comb, y="value", hue = "variable")
plt.xticks(rotation = 90)

In [ ]:
df.groupby("dataset").mean()

# Importance of input size

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_genes", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    for run in range(2):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        for nb_genes in [100, 200, 500, 1000, 1500, 2000, 5000, 8000]:

            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
            print(f"Perc 0 {perc_0}")
            cluster_number = np.unique(Y).shape[0]

            X = train.preprocess(X, nb_genes=nb_genes)
            nb_zeros = int(0.8 * nb_genes)
            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=100,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/inputs/{dataset}_{nb_genes}/")

            #         df.loc[df.shape[0]] = [
            #                 dataset, perc_0, nb_genes, 'kmeans_representation_0',dresults['kmeans_representation_0']
            #             ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, nb_genes, 'leiden_representation_0',
                dresults['leiden_representation_0'], run]

#             pxt = PCA(2).fit_transform(X)
#             dresults["original"] = utils.evaluate(X, Y, cluster_number)[1]
#             dresults["pca"] = utils.evaluate(pxt, Y, cluster_number)[1]
            print(dresults)
    df.to_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df.groupby(["nb_genes"]).mean()

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_genes": "Nb input genes"})

In [ ]:
df["dataset"].unique()


In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size.pdf", bbox_inches='tight')

In [ ]:
datasets = ['10X PBMC (2100)',
       'Mouse ES\nCell (2100)', 'Worm Neuron\nCell (2100)',
       'Mouse Bladder\n Cell (2100)']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size_2100.pdf", bbox_inches='tight')

# Importance of the number of epochs

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_epochs", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    nb_genes = 1500
    for epochs in [5, 50, 100, 300]:

        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
        print(f"Perc 0 {perc_0}")
        cluster_number = np.unique(Y).shape[0]

        X = train.preprocess(X, nb_genes=nb_genes)
        nb_zeros = int(0.8 * nb_genes)
        for run in range(2):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=epochs,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/epochs/{dataset}_{epochs}/")

            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'kmeans_representation_0',
                dresults['kmeans_representation_0'], run
            ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'leiden_representation_0',
                dresults['leiden_representation_0'], run
            ]

            print(dresults)
            df.to_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_epochs": "Nb epochs"})

In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(7, 3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb epochs",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title ="Number of epochs",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_nb_epochs.pdf", bbox_inches='tight')